<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Function Calling with Assistants

There may custom functionality that can't be done by the LLM Assistant, but the LLM assistant is "smart" enough to know how to fill in a Python function for the task. This is where we can provide a custom function that is run on our end and work in conjunction with the assistant. This is a powerful tool, because you now have the power of the OpenAI LLM but with the full flexibility of whatever Python code you can run on your end. Let's work through an example:

## Step 1: Define your custom function

In [3]:
def word_definition_quiz(word,definition_options):
    '''
    INPUTS: 
        word str = A single string representing a word
        definition_options list[str] = A list of 4 potential defintions, where only one is correct!
    OUTPUTS:
        response str = The user selected definition
    '''
    print("Hello! Let's test your knowledge about words.")
    print(f"What is the correct definition of this word: {word}\n")
    
    for num,option in enumerate(definition_options):
        print('\n')
        print(f"Definition #{num} is: {option}")
    
    print('\n')
    num_choice = input("What is your choice? (Return the single number option)")
    
    return definition_options[int(num_choice)]

**Test the function.**

In [4]:
word = "serendipity"
definition_options = [
    "The occurrence of events by chance in a happy or beneficial way.",
    "A feeling of intense fear, shock, or disgust.",
    "A method of painting using opaque pigments ground in water and thickened with a glue-like substance.",
    "The action of delaying or postponing something."
]

# Call the function with these arguments
response = word_definition_quiz(word, definition_options)


Hello! Let's test your knowledge about words.
What is the correct definition of this word: serendipity



Definition #0 is: The occurrence of events by chance in a happy or beneficial way.


Definition #1 is: A feeling of intense fear, shock, or disgust.


Definition #2 is: A method of painting using opaque pigments ground in water and thickened with a glue-like substance.


Definition #3 is: The action of delaying or postponing something.




What is your choice? (Return the single number option) 0


In [5]:
response

'The occurrence of events by chance in a happy or beneficial way.'

## Step 2: Define Custom Function in JSON Format.

Here is the example format that the Python function must be defined in, the most up to date information can be found in the OpenAI Documentation: https://platform.openai.com/docs/assistants/tools/function-calling.


**JSON TOOLS DEFINITION GENERAL FORMAT:**

    tool = {'type':'function',
                'function':{
                    'name': 'my_function_name',
                    'parameters':{
                        "type":"object",
                          "properties":{
                              "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                              "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                          },
                        'required' : ['parameter_one','parameter_two']
                    }

        }
    } 

In [6]:
tool = {'type':'function',
            'function':{
                'name': 'my_function_name',
                'parameters':{
                    "type":"object",
                      "properties":{
                          "parameter_one": {'type':'string','description':"A text description for the LLM of what this parameter should be. Note the JSON type."},
                          "parameter_two":{'type':'integer','description':"A text description for the LLM of this parameter, note the JSON type."}
                      },
                    'required' : ['parameter_one','parameter_two']
                }

    }
} 

In [28]:
# https://community.openai.com/t/function-calling-passing-a-list-of-values/369032
function_json = {'type':'function',
            'function':{
                'name': 'word_definition_quiz',
                'parameters':{
                    "type":"object",
                      "properties":{
                          "word": {'type':'string','description':"A single word"},
                          "definitions_list":{'type':'array',
                                        'items':{'type':'string'},
                                        'description':"A Python list of strings of definitions, with only one correct definition pertaining to the 'word' parameter. Just the string definitions."}
                      },
                    'required' : ["word","definitions_list"]
                }

    }
} 

## Step 3: Create Assistant with Function Calling Tool

In [12]:
from openai import OpenAI
client = OpenAI()

In [13]:
assistant = client.beta.assistants.create(
  instructions="You help create a quiz for checking defintions of words, providing a word and then multiple definition options, where only one option is correct.",
  model="gpt-3.5-turbo",
  tools=[function_json]
)

## Step 4: Create a Thread, Message, and Run

In [14]:
thread = client.beta.threads.create()

In [15]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a new quiz question word and definition list. Then let me know if the student answer recieved is correct.",
)

In [17]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [18]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [19]:
run = wait_on_run(run,thread)

In [20]:
run.status

'requires_action'

## Step 5: Take Run Results and Call Function Locally

In [22]:
import json

# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: word_definition_quiz
Function Arguments:


{'word': 'abate',
 'definition': '1. to become less intense or widespread\n2. to postpone or delay\n3. to make stronger or more intense'}

In [27]:
answer = word_definition_quiz(arguments['word'],arguments['definition'])

Hello! Let's test your knowledge about words.
What is the correct definition of this word: abate



Definition #0 is: 1. to become less intense or widespread


Definition #1 is: 2. to postpone or delay


Definition #2 is: 3. to make stronger or more intense




What is your choice? (Return the single number option) 1


In [29]:
answer

'2. to postpone or delay'

## Step 6: Return Local Function Response to the Tool in the Assistant

In [30]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(answer),
        }
    ],
)

In [31]:
run = wait_on_run(run, thread)

In [32]:
run.status

'completed'

## Step 7: Retrieve Messages

In [35]:
messages = client.beta.threads.messages.list(thread.id)

In [38]:
for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

The student answer received is incorrect.


Create a new quiz question word and definition list. Then let me know if the student answer recieved is correct.




## Optional: Delete Assistant

In [39]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_7cVv7JptjJuNQWiA8mTTeDMb', deleted=True, object='assistant.deleted')
